# 循環神經網路

在上一個模組中，我們探討了文本的豐富語義表示。我們使用的架構能夠捕捉句子中單詞的聚合意義，但它並未考慮單詞的**順序**，因為嵌入後的聚合操作會將原始文本中的這些信息移除。由於這些模型無法表示單詞的順序，因此它們無法解決更複雜或更具歧義的任務，例如文本生成或問題回答。

為了捕捉文本序列的意義，我們將使用一種稱為**循環神經網路**（Recurrent Neural Network，簡稱 RNN）的神經網路架構。在使用 RNN 時，我們會將句子逐個標記（token）傳遞給網路，網路會生成某種**狀態**，然後我們將該狀態與下一個標記一起再次傳遞給網路。

![顯示循環神經網路生成示例的圖片。](../../../../../lessons/5-NLP/16-RNN/images/rnn.png)

給定輸入標記序列 $X_0,\dots,X_n$，RNN 會創建一個神經網路區塊的序列，並通過反向傳播對該序列進行端到端訓練。每個網路區塊將一對 $(X_i,S_i)$ 作為輸入，並生成 $S_{i+1}$ 作為結果。最終狀態 $S_n$ 或輸出 $Y_n$ 會進入線性分類器以生成結果。所有網路區塊共享相同的權重，並通過一次反向傳播訓練完成端到端的學習。

> 上圖展示了循環神經網路的展開形式（左側）以及更緊湊的循環表示形式（右側）。需要注意的是，所有 RNN 單元都具有相同的**可共享權重**。

由於狀態向量 $S_0,\dots,S_n$ 是通過網路傳遞的，RNN 能夠學習單詞之間的順序依賴性。例如，當單詞 *not* 出現在序列中的某處時，它可以學會在狀態向量中否定某些元素。

在內部，每個 RNN 單元包含兩個權重矩陣：$W_H$ 和 $W_I$，以及偏置 $b$。在每個 RNN 步驟中，給定輸入 $X_i$ 和輸入狀態 $S_i$，輸出狀態的計算方式為 $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$，其中 $f$ 是一個激活函數（通常是 $\tanh$）。

> 對於像文本生成（我們將在下一單元中討論）或機器翻譯這樣的問題，我們還希望在每個 RNN 步驟中獲得一些輸出值。在這種情況下，還會有另一個矩陣 $W_O$，輸出值的計算方式為 $Y_i=f(W_O\times S_i+b_O)$。

現在讓我們看看循環神經網路如何幫助我們對新聞數據集進行分類。

> 在沙盒環境中，我們需要運行以下程式碼單元，以確保安裝了所需的庫並預取了數據。如果您在本地運行，則可以跳過以下程式碼單元。


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

在訓練大型模型時，GPU 記憶體分配可能會成為一個問題。我們也可能需要嘗試不同的迷你批次大小，以便數據能夠適配 GPU 記憶體，同時確保訓練速度足夠快。如果您在自己的 GPU 機器上運行此代碼，可以嘗試調整迷你批次大小來加快訓練速度。

> **注意**: 某些版本的 NVidia 驅動程式已知在訓練模型後不會釋放記憶體。我們在這個筆記本中運行了幾個範例，這可能會導致在某些配置中記憶體耗盡，特別是如果您在同一個筆記本中進行自己的實驗時。如果在開始訓練模型時遇到一些奇怪的錯誤，您可能需要重新啟動筆記本的內核。


In [3]:
batch_size = 16
embed_size = 64

## 簡單的 RNN 分類器

在簡單的 RNN 中，每個循環單元都是一個簡單的線性網絡，它接收輸入向量和狀態向量，並生成新的狀態向量。在 Keras 中，可以使用 `SimpleRNN` 層來表示。

雖然我們可以直接將 one-hot 編碼的標記傳遞給 RNN 層，但由於其高維度性，這並不是一個好主意。因此，我們將使用嵌入層來降低詞向量的維度，接著是 RNN 層，最後是一個 `Dense` 分類器。

> **注意**：在維度不太高的情況下，例如使用字符級標記化時，直接將 one-hot 編碼的標記傳遞給 RNN 單元可能是合理的選擇。


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **注意：** 為了簡化，我們在這裡使用未經訓練的嵌入層，但如果想要更好的結果，可以使用 Word2Vec 預訓練的嵌入層，如前一單元所述。你可以嘗試將此程式碼改寫為使用預訓練嵌入層，這會是一個很好的練習。

現在讓我們來訓練 RNN。一般來說，RNN 的訓練相當困難，因為當 RNN 單元沿著序列長度展開時，涉及反向傳播的層數會非常多。因此，我們需要選擇較小的學習率，並在更大的數據集上訓練網路以獲得良好的結果。這可能需要相當長的時間，因此建議使用 GPU。

為了加快速度，我們將僅使用新聞標題來訓練 RNN 模型，省略描述部分。你可以嘗試使用描述進行訓練，看看是否能讓模型成功訓練。


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **注意** 這裡的準確性可能會較低，因為我們僅針對新聞標題進行訓練。


## 重新探討變長序列

請記住，`TextVectorization` 層會自動在小批量中用填充標記（pad tokens）填充變長序列。然而，這些填充標記也會參與訓練，這可能會使模型的收斂變得更加複雜。

我們可以採取幾種方法來減少填充的數量。其中一種方法是根據序列長度重新排序數據集，並將所有序列按大小分組。這可以使用 `tf.data.experimental.bucket_by_sequence_length` 函數來完成（參見[文件](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)）。

另一種方法是使用**遮罩（masking）**。在 Keras 中，一些層支持額外的輸入，用於指示哪些標記應該在訓練時被考慮。要將遮罩整合到模型中，我們可以選擇加入一個單獨的 `Masking` 層（[文件](https://keras.io/api/layers/core_layers/masking/)），或者在 `Embedding` 層中指定參數 `mask_zero=True`。

> **Note**: 完成整個數據集的一個訓練週期大約需要 5 分鐘。如果你失去耐心，可以隨時中斷訓練。你還可以通過在 `ds_train` 和 `ds_test` 數據集後添加 `.take(...)` 子句來限制用於訓練的數據量。


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


現在我們使用遮罩技術，可以在整個標題和描述的數據集上訓練模型。

> **注意**：你是否注意到我們一直在使用基於新聞標題訓練的向量化工具，而不是整篇文章的內容？這可能會導致某些詞元被忽略，因此重新訓練向量化工具會更好。不過，這可能只會帶來非常小的影響，所以為了簡化流程，我們將繼續使用之前預訓練的向量化工具。


## LSTM: 長短期記憶

RNN 的主要問題之一是**梯度消失**。RNN 可能會非常長，並且在反向傳播過程中，可能很難將梯度傳遞回網路的第一層。當這種情況發生時，網路無法學習遠距離的詞元之間的關係。為了避免這個問題，可以通過使用**門控機制**引入**顯式狀態管理**。最常見的兩種引入門控機制的架構是**長短期記憶**（LSTM）和**門控循環單元**（GRU）。我們在這裡將介紹 LSTM。

![顯示長短期記憶單元範例的圖片](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM 網路的組織方式與 RNN 類似，但有兩個狀態會從一層傳遞到下一層：實際狀態 $c$ 和隱藏向量 $h$。在每個單元中，隱藏向量 $h_{t-1}$ 與輸入 $x_t$ 結合，並共同控制狀態 $c_t$ 和輸出 $h_{t}$ 的變化，這是通過**門控機制**實現的。每個門都有 sigmoid 激活函數（輸出範圍為 $[0,1]$），可以將其視為與狀態向量相乘的位掩碼。LSTM 包含以下門控機制（如上圖從左到右）：
* **遺忘門**：決定向量 $c_{t-1}$ 的哪些部分需要遺忘，哪些需要保留。
* **輸入門**：決定來自輸入向量和前一隱藏向量的信息有多少應該被整合到狀態向量中。
* **輸出門**：接收新的狀態向量，並決定其哪些部分將用於生成新的隱藏向量 $h_t$。

狀態 $c$ 的組成部分可以被視為可以開啟或關閉的標誌。例如，當我們在序列中遇到名字 *Alice* 時，我們猜測它指的是一位女性，並在狀態中設置一個標誌，表示句子中有一個女性名詞。當我們進一步遇到單詞 *and Tom* 時，我們會設置一個標誌，表示句子中有一個複數名詞。因此，通過操作狀態，我們可以追蹤句子的語法屬性。

> **Note**: 這裡有一個很棒的資源可以幫助理解 LSTM 的內部結構：[Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) by Christopher Olah。

雖然 LSTM 單元的內部結構看起來可能很複雜，但 Keras 將這些實現隱藏在 `LSTM` 層中，因此在上面的範例中，我們只需要替換循環層即可：


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## 雙向與多層 RNN

在我們之前的範例中，循環神經網路都是從序列的開頭運算到結尾。這對我們來說很自然，因為它遵循了我們閱讀或聆聽語音的方向。然而，對於需要隨機存取輸入序列的情境，讓循環運算在兩個方向上進行會更合理。允許在兩個方向上進行運算的 RNN 被稱為 **雙向** RNN，可以透過將循環層包裹在特殊的 `Bidirectional` 層中來建立。

> **Note**: `Bidirectional` 層會在其內部建立該層的兩個副本，並將其中一個副本的 `go_backwards` 屬性設置為 `True`，使其沿著序列的相反方向運算。

無論是單向還是雙向的循環神經網路，都能捕捉序列中的模式，並將其存儲到狀態向量中或作為輸出返回。與卷積神經網路類似，我們可以在第一層之後再建立另一個循環層，以捕捉更高層次的模式，這些模式是由第一層提取的低層次模式構建而成的。這引出了 **多層 RNN** 的概念，它由兩層或更多層循環神經網路組成，其中前一層的輸出作為下一層的輸入。

![顯示多層長短期記憶 RNN 的圖片](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*圖片來源：[這篇精彩文章](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) 作者 Fernando López。*

Keras 讓構建這些網路變得非常簡單，因為你只需要在模型中添加更多的循環層。對於除了最後一層以外的所有層，我們需要指定 `return_sequences=True` 參數，因為我們需要該層返回所有中間狀態，而不僅僅是循環運算的最終狀態。

現在我們來為分類問題構建一個雙層雙向 LSTM。

> **Note** 這段程式碼執行時間可能會比較長，但它提供了我們目前看到的最高準確率。所以也許值得等待並查看結果。


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNN 用於其他任務

到目前為止，我們專注於使用 RNN 來對文本序列進行分類。但它們還可以處理更多任務，例如文本生成和機器翻譯——我們將在下一單元中探討這些任務。



---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原始語言的文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤讀概不負責。
